# Introducción

En el presente proyecto, abordamos el análisis y modelado predictivo de la **Encuesta Anual de Hogares 2019** de Argentina, con el objetivo de extraer información relevante sobre las características socioeconómicas de los hogares y construir modelos que permitan predecir el ingreso total familiar. Este estudio resulta clave para comprender la distribución de recursos, identificar factores asociados al bienestar económico y aportar herramientas de apoyo a la toma de decisiones en ámbitos sociales y de políticas públicas.

---

## Objetivos

1. **Explorar** las variables demográficas y de ingreso para detectar patrones, sesgos y relaciones relevantes.  
2. **Preprocesar** los datos: limpieza, tratamiento de valores faltantes, codificación de variables categóricas y detección de outliers.  
3. **Desarrollar** modelos de regresión y clasificación que permitan:
   - Estimar el ingreso total de un hogar (regresión).  
   - Clasificar hogares según su situación económica (por ejemplo, por debajo o por encima de la línea de pobreza).  
4. **Evaluar** el desempeño de los modelos mediante métricas apropiadas (MSE, R², accuracy, AUC) y comparar técnicas (Regresión Lineal vs. Random Forest, Regresión Logística vs. XGBoost).  
5. **Presentar** hallazgos y conclusiones, así como recomendaciones basadas en los resultados obtenidos.

---

## Estructura del Informe

1. **Introducción** – Contexto, datos y objetivos.  
2. **Metodología** – Fuentes de datos, limpieza y transformación.  
3. **Análisis Exploratorio** – Visualizaciones y descubrimiento de patrones.  
4. **Modelado** – Definición de variables, entrenamiento y validación de modelos.  
5. **Resultados** – Comparación de métricas y selección del mejor modelo.  
6. **Conclusiones y Trabajo Futuro** – Interpretación de resultados, limitaciones y posibles extensiones.

---


# Paso 1: Importar librerías y cargar datos

En este bloque importamos las librerías necesarias y cargamos el CSV en un DataFrame de pandas.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carga del dataset (ajusta el nombre si cambiaste la ruta)
ruta = "/content/encuesta-anual-hogares-2019.csv"
df = pd.read_csv(ruta, encoding='latin-1')

# Paso 2: Inspección inicial del DataFrame

Aquí vemos el tamaño del DataFrame, un vistazo a las primeras filas y la información general de columnas y tipos.


In [8]:
# Dimensiones (filas, columnas)
print("Dimensiones:", df.shape)

Dimensiones: (14319, 31)


In [14]:
# Primeras 5 filas
df.head()

,id,nhogar,miembro,comuna,dominio,edad,sexo,parentesco_jefe,situacion_conyugal,num_miembro_padre,...,ingreso_per_capita_familiar,estado_educativo,sector_educativo,nivel_actual,nivel_max_educativo,años_escolaridad,lugar_nacimiento,afiliacion_salud,hijos_nacidos_vivos,cantidad_hijos_nac_vivos
0,1,1,1,5,Resto de la Ciudad,18,Mujer,Jefe,Soltero/a,Padre no vive en el hogar,...,9000,Asiste,Estatal/publico,Universitario,Otras escuelas especiales,12,PBA excepto GBA,Solo obra social,No,No corresponde
1,1,1,2,5,Resto de la Ciudad,18,Mujer,Otro no familiar,Soltero/a,Padre no vive en el hogar,...,9000,Asiste,Estatal/publico,Universitario,Otras escuelas especiales,12,Otra provincia,Solo plan de medicina prepaga por contratación...,No,No corresponde
2,2,1,1,2,Resto de la Ciudad,18,Varon,Jefe,Soltero/a,Padre no vive en el hogar,...,33333,Asiste,Privado religioso,Universitario,Otras escuelas especiales,12,CABA,Solo plan de medicina prepaga por contratación...,NaN,No corresponde
3,2,1,2,2,Resto de la Ciudad,50,Mujer,Padre/Madre/Suegro/a,Viudo/a,No corresponde,...,33333,No asiste pero asistió,No corresponde,No corresponde,Secundario/medio comun,17,CABA,Solo prepaga o mutual via OS,Si,2
4,2,1,3,2,Resto de la Ciudad,17,Varon,Otro familiar,Soltero/a,Padre no vive en el hogar,...,33333,Asiste,Privado religioso,Secundario/medio comun,EGB (1° a 9° año),10,CABA,Solo plan de medicina prepaga por contratación...,NaN,No corresponde


In [15]:
# Tipos de columnas y conteo de nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14319 entries, 0 to 14318
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           14319 non-null  int64 
 1   nhogar                       14319 non-null  int64 
 2   miembro                      14319 non-null  int64 
 3   comuna                       14319 non-null  int64 
 4   dominio                      14319 non-null  object
 5   edad                         14319 non-null  int64 
 6   sexo                         14319 non-null  object
 7   parentesco_jefe              14319 non-null  object
 8   situacion_conyugal           14318 non-null  object
 9   num_miembro_padre            14319 non-null  object
 10  num_miembro_madre            14319 non-null  object
 11  estado_ocupacional           14319 non-null  object
 12  cat_ocupacional              14319 non-null  object
 13  calidad_ingresos_lab         14

# Paso 3: Estadísticas descriptivas por tipo

Obtenemos estadísticas de variables numéricas y un resumen de variables categóricas.

In [22]:
# Variables numéricas
df.describe().T.round(1)

,count,mean,std,min,25%,50%,75%,max
id,14319.0,2710.0,1535.9,1.0,1473.0,2611.0,3898.0,5795.0
nhogar,14319.0,1.0,0.1,1.0,1.0,1.0,1.0,7.0
miembro,14319.0,2.1,1.4,1.0,1.0,2.0,3.0,19.0
comuna,14319.0,7.6,4.2,1.0,4.0,8.0,11.0,15.0
edad,14319.0,38.8,23.1,0.0,20.0,37.0,57.0,100.0
ingreso_total_lab,14319.0,20078.6,34698.2,0.0,0.0,2500.0,30000.0,1000000.0
ingreso_total_no_lab,14319.0,6016.2,16065.4,0.0,0.0,0.0,4000.0,500000.0
ingresos_totales,14319.0,26094.9,37152.5,0.0,0.0,16000.0,37000.0,1000000.0
ingresos_familiares,14319.0,70212.8,62685.7,0.0,30000.0,54000.0,90000.0,1000000.0
ingreso_per_capita_familiar,14319.0,26192.0,27463.9,0.0,10500.0,19900.0,33500.0,1000000.0


In [28]:
# Variables categóricas
display(df.describe(include=['object']).T)

,count,unique,top,freq
dominio,14319,2,Resto de la Ciudad,12580
sexo,14319,2,Mujer,7723
parentesco_jefe,14319,9,Jefe,5848
situacion_conyugal,14318,7,Soltero/a,3542
num_miembro_padre,14319,9,No corresponde,9817
num_miembro_madre,14319,11,No corresponde,9817
estado_ocupacional,14319,3,Ocupado,7387
cat_ocupacional,14319,5,No corresponde,6932
calidad_ingresos_lab,14319,4,Tuvo ingresos y declara monto,5855
calidad_ingresos_no_lab,14319,4,No tuvo ingresos,8596


# Paso 4: Conteo de valores faltantes

Identificamos cuántos valores nulos hay en cada columna para planear su tratamiento.


In [29]:
df.isnull().sum().sort_values(ascending=False)

,0
hijos_nacidos_vivos,7784
nivel_max_educativo,1054
años_escolaridad,62
afiliacion_salud,4
sector_educativo,3
lugar_nacimiento,1
situacion_conyugal,1
sexo,0
parentesco_jefe,0
nhogar,0


# Paso 5: Eliminación de duplicados y cálculo de porcentaje de valores faltantes

En este bloque eliminamos filas duplicadas y calculamos el porcentaje de nulos por columna para planear nuestra estrategia de imputación.


In [30]:
# 5.1 Quitar duplicados
df = df.drop_duplicates()

# 5.2 Calcular porcentaje de valores faltantes por columna
missing_pct = df.isnull().mean().sort_values(ascending=False) * 100
print("Porcentaje de nulos por columna:\n", missing_pct)


Porcentaje de nulos por columna:
 hijos_nacidos_vivos            54.361338
nivel_max_educativo             7.360849
años_escolaridad                0.432991
afiliacion_salud                0.027935
sector_educativo                0.020951
lugar_nacimiento                0.006984
situacion_conyugal              0.006984
sexo                            0.000000
parentesco_jefe                 0.000000
nhogar                          0.000000
id                              0.000000
comuna                          0.000000
dominio                         0.000000
edad                            0.000000
miembro                         0.000000
ingreso_total_lab               0.000000
calidad_ingresos_lab            0.000000
cat_ocupacional                 0.000000
estado_ocupacional              0.000000
num_miembro_madre               0.000000
num_miembro_padre               0.000000
calidad_ingresos_no_lab         0.000000
ingreso_total_no_lab            0.000000
estado_educativo       

# Paso 6: Estrategia de tratamiento de nulos

Como ejemplo:
- Eliminaremos columnas con más de 30 % de datos faltantes.  
- Rellenaremos el resto:  
  - Variables numéricas con la mediana.  
  - Variables categóricas con la moda.


In [31]:
# 6.1 Eliminar columnas > 30 % nulos
cols_to_drop = missing_pct[missing_pct > 30].index
df = df.drop(columns=cols_to_drop)

# 6.2 Identificar columnas numéricas y categóricas
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns

# 6.3 Rellenar numéricas con mediana
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# 6.4 Rellenar categóricas con moda
for c in cat_cols:
    df[c] = df[c].fillna(df[c].mode()[0])

print("Tratamiento de nulos completado. Nuevos nulos:\n", df.isnull().sum())


Tratamiento de nulos completado. Nuevos nulos:
 id                             0
nhogar                         0
miembro                        0
comuna                         0
dominio                        0
edad                           0
sexo                           0
parentesco_jefe                0
situacion_conyugal             0
num_miembro_padre              0
num_miembro_madre              0
estado_ocupacional             0
cat_ocupacional                0
calidad_ingresos_lab           0
ingreso_total_lab              0
calidad_ingresos_no_lab        0
ingreso_total_no_lab           0
calidad_ingresos_totales       0
ingresos_totales               0
calidad_ingresos_familiares    0
ingresos_familiares            0
ingreso_per_capita_familiar    0
estado_educativo               0
sector_educativo               0
nivel_actual                   0
nivel_max_educativo            0
años_escolaridad               0
lugar_nacimiento               0
afiliacion_salud            